# Création des instances pièges


L'objectif est de regarder les comportements aux limites de l'optimiseur lorsqu'il travaille sur des instances spécialement créées pour être difficile à traiter.


Comment créer une instance piège : 

- rendre difficile l'optimisation selon le critère 3 : proposer des projets qui ne sont pas réalisables en 1 jour


## Format d'une instance

Pour créer une instance il faut définir : 

- un horizon de temps *horizon* : entier naturel dans [1,limite_horizon]
- un nombre de qualifications *nb_qualifs* : entier naturel dans [1,limite_qualifs]
- une population de personnel : 
    - un nombre d'employé *nb_staff* : entier naturel dans [1,limite_staff]
    
    
    Puis pour chaque employé : 
    - un nom unique *name* : entier naturel représentant l'ID de l'employé, de [1,nb_staff]
    - un ensemble de qualification non vide *qualifications* : sous-liste de la liste des qualifications construite en amont
    - un ensemble de jours de vacances *vacations* : un sous ensemble de [0,horizon]
- une population de projet : 
    - un nombre de projet *nb_jobs* : entier naturel dans [1,limite_jobs]
    
    
    Puis pour chaque projet :
    - un nom unique *name* : entier naturel représentant l'ID du projet, de [1,nb_jobs]
    - un ensemble de compétences concernées, avec un nombre de jours à travailler par compétence *working_days_per_qualifications* : dictionnaire (compétence, jours), avec jours dans [1,limite_jour_comp_job]
    - une deadline *due_date* : entier naturel dans [1,horizon]
    - un gain *gain* : entier naturel dans [1,10\*horizon]
    - une pénalité de retard journalière *daily_penalty* : entier naturel de [0,gain]

In [73]:
# Bibliothèques

import json
import random as rd
from os.path import join, exists


In [74]:
class DummyRandomSchedulingDataset: 
    """Instances du jeu de données créées uniquement à partir des spécifications d'une instance valide.
    
    Il n'y a rien qui est fait pour piéger l'optimizer Gurobi pour le moment.
    """
    
    # Constantes de définition
    LIMITE_HORIZON = 100
    # On définit les autres limites par rapport à horizon plutôt qu'à LIMITE_HORIZON pour avoir des instances cohérentes
    # TODO: Hypothèse à discuter
    # LIMITE_QUALIFS = limite_horizon
    # LIMITE_STAFF = limite_horizon
    # LIMITE_JOBS = 2*limite_horizon
    # LIMITE_GAIN = 10*limite_horizon
    
    DEFAULT_FOLDER_PATH = '.\\instances\\instances_created'
        
    def __init__(self, limite_horizon=LIMITE_HORIZON):
        
        # Initialisation des paramètres aléatoires
        self.horizon = rd.randrange(2, limite_horizon + 1) # Les jours planifiés seront indexés dans [0,horizon-1]
        self.nb_staff = rd.randrange(2, self.horizon + 1)
        self.nb_qualifs = rd.randrange(2, self.nb_staff + 1)
        self.nb_jobs = rd.randrange(self.nb_staff, 2 * self.horizon + 1)
        self.gain_max = 10 * self.horizon
        self.max_vacations = max(1, self.horizon // 10)
        self.max_jour_consultant = self.horizon * self.nb_staff # Le nombre total de jour/consultant sur l'ensemble de l'horizon
        
        # On définit un nombre de jour/consultants acceptable pour un projet qui a un gain de gain_max
        # On veut que si on multiplie cette constante par nb_jobs on trouve un résultat environ 3-5 fois supérieur à max_jour_consultant
        # On choisira donc complètement arbitrairement 4
        # Me semble pertinent car si on calcule la somme des jours/consultants de chaque projet, on devrait trouver un résultat environ 2 fois supérieur à max_jour_consultants
        self.travail_pour_gain_max = 4 * self.max_jour_consultant / self.nb_jobs
        
        # Définition de la liste de classification
        self.qualifications = [str(qualif) for qualif in range(1, self.nb_qualifs + 1)]
        
        # Définition du personnel
        self.staff = []
        for staff_index in range(self.nb_staff + 1):
            nb_qualifs_employee = rd.randrange(1, self.nb_qualifs + 1)
            nb_vacations_employee = rd.randrange(self.max_vacations)
            employee = {
                'name': f'Consultant{staff_index + 1}',
                'qualifications': rd.sample(self.qualifications, nb_qualifs_employee),
                'vacations': rd.sample(range(self.horizon), nb_vacations_employee)
            }
            self.staff.append(employee)
        
        # FIXME: faire en sorte que chaque qualification possible soit au moins réalisée par 1 membre du personnel
        
        # Définition des projets
        self.jobs = []
        for job_index in range(self.nb_jobs + 1):
            gain = rd.randrange(1, self.gain_max + 1)
            
            # Calculer le nb de jours totals à travailler sur le projet, le rendre fonction du ratio gain / gain_max
            gain_ratio = gain / self.gain_max
            mean_jour_consultant = gain_ratio * self.travail_pour_gain_max # on veut prendre une fourchette autour de ça plutôt que le chiffre exact
            noise_coef = 0.3 # On choisit de prendre une fourchette de 30% autour de la moyenne
            working_days = rd.randrange(max(1, round((1 - noise_coef) * mean_jour_consultant)), max(2, round((1 + noise_coef) * mean_jour_consultant) + 1))
            
            # Une fois qu'on a le chiffre, on choisit aléatoirement un sous-ensemble de compétences à remplir
            job_nb_qualifs = rd.randrange(1, min(working_days, len(self.qualifications)) + 1)
            job_qualifs = rd.sample(self.qualifications, job_nb_qualifs)
            
            # Puis on réparti aléatoirement les jours de travail entre les compétences choisies, en faisant en sorte que chacune ait au moins un jour travaillé
            # D'abord on affecte un point à chaque qualif et on soustrait le nombre de points à affecter correspondant
            working_days_per_qualification = {qualif:1 for qualif in job_qualifs}
            remaining_wd = working_days - job_nb_qualifs
            while remaining_wd > 0:
                qualif = rd.choice(job_qualifs)
                working_days_per_qualification[qualif] += 1
                remaining_wd -= 1
            
            job = {
                'name': f'Job{job_index + 1}',
                'gain': gain,
                'due_date': rd.randrange(self.horizon),
                'daily_penalty': rd.randrange(gain + 1),
                'working_days_per_qualification': working_days_per_qualification
            }
            
            self.jobs.append(job)
        
    def to_json(self, filename, instance_folder_path=DEFAULT_FOLDER_PATH):
        """Create the dict with correct format from instance object and transform it to json. 
        
        Result file is stored under the given filename at the given path.
        """
        instance_dict = {
            'horizon': self.horizon, 
            'qualifications': self.qualifications,
            'staff': self.staff, 
            'jobs': self.jobs
        }
        if filename[-5:] != '.json':
            filename += '.json'
        path = join(instance_folder_path, filename)
        if exists(path):
            print('\nYour file failed to save because this path already exists.\n')
            return None
        with open(path, 'a') as result_file:
            json.dump(instance_dict, result_file)
            


In [75]:
dummy_test = DummyRandomSchedulingDataset()

In [76]:
#dummy_test.horizon

In [77]:
#dummy_test.qualifications

In [78]:
#dummy_test.staff

In [79]:
#dummy_test.jobs

In [80]:
dummy_test.to_json('dummy_test')